In [7]:
import spacy
import pandas as pd
import plotly.express as pe
import openpyxl
from io import BytesIO
from pyxlsb import open_workbook as open_xlsb
import xlsxwriter

In [64]:
nlp=spacy.load("output/model-last/")

In [8]:
df=pd.read_excel("registo_consultas_teste.xlsx")

In [48]:
df.head()

,ID,Nota
0,1,Doente bem. Está actualmente a fazer Enzalutam...
1,2,"Hoje vem apenas o marido. Doente mal, em progr..."
2,3,Tudo ok. Vai manter trifluridina+tipiracilo 1x...
3,4,Vem a própria. Refere cefaleias e insónias oca...
4,5,"A fazer Osimertinib 80mg/dia. Animada, sem pro..."


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      5 non-null      int64 
 1   Nota    5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [9]:
df.ID.astype(float)


0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: ID, dtype: float64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      5 non-null      int64 
 1   Nota    5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [71]:
ent = []
labels = []
ID = []
def processar(x):
    doc = nlp(x["Nota"])
    for entity in doc.ents:
        ID.append(x["ID"])
        ent.append(entity.text)
        labels.append(entity.label_)

In [72]:
for i in range(len(df.index)):
    processar(df.iloc[i,:])

In [53]:
base = pd.DataFrame({"ID":ID,"Entidade":ent,"Classificação":labels})
base

,ID,Entidade,Classificação
0,1,bem,Estado
1,1,Enzalutamida,Terapêutica
2,1,rash,RAM
3,1,TAC,MCDT
4,2,mal,Estado
5,2,em progressão,Estado
6,2,capecitabina,Terapêutica
7,2,1500mg,Dose
8,2,bid,Posologia
9,2,diarreia,RAM


In [21]:
RAMS=base.loc[base.Classificação=="RAM"]
pe.pie(RAMS, names=RAMS.Entidade, title = "Distribuição de RAMS", width = 400, height = 400,color_discrete_sequence=pe.colors.qualitative.Vivid)

In [10]:
doc=nlp(texto)

NameError: name 'texto' is not defined

In [ ]:
from spacy import displacy
colors = {"Estado" : "#5F9EA0", "Terapêutica" : "#00FF00","RAM" : "#FFA07A","MCDT" : "#FF00FF", "Dose" : "#696969", "Posologia" : "#E6E6FA"}
options = {"colors" : colors}
displacy.render(doc,style="ent",jupyter="True", options = options)

In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as pe
import spacy
import openpyxl
from spacy import displacy
from io import BytesIO
from pyxlsb import open_workbook as open_xlsb
import xlsxwriter

def text_function(texto):
    nlp = spacy.load("output/model-last/")
    doc = nlp(texto)
    colors = {"Estado" : "#5F9EA0", "Terapêutica" : "#00FF00","RAM" : "#FFA07A","MCDT" : "#FF00FF", "Dose" : "#696969", "Posologia" : "#E6E6FA"}
    options = {"colors" : colors}
    html = displacy.render(doc,style = "ent", jupyter = False,options = options)
    st.markdown(html, unsafe_allow_html = True)

ent = []
labels = []
ID = []
def processar(x):
    nlp = spacy.load("output/model-last/")
    doc = nlp(x["Nota"])
    for entity in doc.ents:
        ID.append(x["ID"])
        ent.append(entity.text)
        labels.append(entity.label_)

def to_excel(w):
    output = BytesIO()
    writer = pd.ExcelWriter(output, engine='xlsxwriter')
    w.to_excel(writer, index=False, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    format1 = workbook.add_format({'num_format': '0'}) 
    worksheet.set_column('A:A', None, format1)  
    writer.save()
    processed_data = output.getvalue()
    return processed_data

def main():
    if 'formato' not in st.session_state:
        st.session_state['formato'] = 'RAMS'
    elif st.session_state["formato"] == "RAMS":
        st.session_state["formato"] = "RAMS"
    elif st.session_state["formato"] == "Terapêutica":
        st.session_state["formato"] = "Terapêutica"
    else:
        st.session_state["formato"] = "Estado"
    st.set_page_config(layout = 'wide', initial_sidebar_state = 'expanded')
    st.title('NLP em consulta farmacêutica')
    
    st.header("Utilizando um ficheiro")
    ficheiro = st.file_uploader("Faça Upload do ficheiro contendo as notas clínicas", type=["xls","xlsx"])
    if ficheiro is not None:
        file_data =  ficheiro.read()
        df=pd.read_excel(ficheiro)
        df["ID"] = df["ID"].astype(int)
        df["Nota"] = df["Nota"].astype(str)

        
    st.header("Utilizando texto livre")
    texto = st.text_input("Insira aqui a nota")

    
    if ficheiro is not None:
        st.header("Dashboard de análise")
        st.write("Foram analisados registos de **{}** doentes".format(df.ID.nunique()))
        for i in range(len(df.index)):
            processar(df.iloc[i,:])
        base = pd.DataFrame({"ID":ID,"Entidade":ent,"Classific":labels})
        RAMS=base.loc[base.Classific=="RAM"]
        fig1=pe.pie(RAMS, names=RAMS.Entidade, title = "Distribuição de RAMS", width = 400, height = 400, color_discrete_sequence=pe.colors.qualitative.Vivid)
        Tx=base.loc[base.Classific=="Terapêutica"]
        fig2=pe.pie(Tx, names=Tx.Entidade, title = "Distribuição de Terapêutica", width = 400, height = 400, color_discrete_sequence=pe.colors.qualitative.Vivid)
        Estado=base.loc[base.Classific=="Estado"]
        fig3=pe.pie(Tx, names=Estado.Entidade, title = "Distribuição de Estado Global", width = 400, height = 400, color_discrete_sequence=pe.colors.qualitative.Vivid)
        col1,col2,col3 = st.columns(3)
        with col1:
            st.write(fig1)
        with col2:
            st.write(fig2)
        with col3:
            st.write(fig3)
            
        st.dataframe(base)
        st.subheader("Exportar relatório de análise")
        formato = st.selectbox("O que quer recolher?",("RAMS","Estado","Terapêutica"),key = "formato")
        if formato == "RAMS":
            ficheiro_download = RAMS
        elif formato == "Estado":
            ficheiro_download = Estado
        else:
            ficheiro_download = Tx
        ficheiro_relatorio = to_excel(ficheiro_download)
        st.download_button(label="Exportar",data=ficheiro_relatorio, file_name="relatorio.xlsx")

    else:
        return text_function(texto)

if __name__ == "__main__":
    main()

Overwriting app.py
